In [14]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from csep import load_stochastic_event_set, load_catalog

In [15]:
project_root = '/Users/wsavran/Projects/CSEP2/u3etas_simulations/landers_experiment'
filename = os.path.join(project_root, '10-23-2018_landers-pt1/results_complete.bin')

u3catalogs = load_stochastic_event_set(type='ucerf3',
                        format='native', filename=filename, name='UCERF3-ETAS')

filtered = []
t0 = time.time()
for u3catalog in u3catalogs:
    filtered.append(u3catalog.filter('magnitude > 3.95'))
t1 = time.time()
print('Filtered {} catalogs in {} seconds.'.format(u3catalog.catalog_id+1, t1-t0))

Filtered 10000 catalogs in 11.800339937210083 seconds.


In [16]:
# Comcat Catalog
epoch_time = 709732655000
duration_in_years = 1.0
t0 = time.time()
comcat = load_catalog(type='comcat', format='native',
                        start_epoch=epoch_time, duration_in_years=1.0,
                        min_magnitude=2.55,
                        min_latitude=31.50, max_latitude=43.00,
                        min_longitude=-125.40, max_longitude=-113.10,
                    name='Comcat')
comcat = comcat.filter('magnitude > 3.95')
comcat_count = comcat.get_number_of_events()
t1 = time.time()
# Statements about Comcat Downloads
print("Fetched Comcat catalog in {} seconds.\n".format(t1-t0))
print("Comcat catalog has the following parameters")
print("Start Date: {}\nEnd Date: {}".format(str(comcat.start_time), str(comcat.end_time)))
print("Min Latitude: {} and Max Latitude: {}".format(comcat.min_latitude, comcat.max_latitude))
print("Min Longitude: {} and Max Longitude: {}".format(comcat.min_longitude, comcat.max_longitude))
print("Min Magnitude: {}\n".format(comcat.min_magnitude))

Fetched Comcat catalog in 5.207682847976685 seconds.

Comcat catalog has the following parameters
Start Date: 1992-06-28 12:00:59.904000+00:00
End Date: 1993-05-31 08:55:11.616000+00:00
Min Latitude: 32.242000579833984 and Max Latitude: 41.52033233642578
Min Longitude: -124.40650177001953 and Max Longitude: -113.19066619873047
Min Magnitude: 3.9600000381469727



In [17]:
# get dataframe representation for all catalogs
f = lambda x: x.get_dataframe()
t0 = time.time()
df = pd.concat(list(map(f, filtered)))
t1 = time.time()
print('Converted {} ruptures into a single DataFrame in {} seconds.'.format(len(df), t1-t0))

Converted 3252898 ruptures into a single DataFrame in 57.2513108253479 seconds.


In [18]:
# get counts, cumulative_counts, percentiles in weekly intervals
df_comcat = comcat.get_dataframe()

# get statistics from stochastic event set
df1 = df.groupby([df['catalog_id'], pd.Grouper(freq='W')])['counts'].agg(['sum'])
df1['cum_sum'] = df1.groupby(level=0).cumsum()
df2 = df1.groupby('datetime').describe(percentiles=(0.05,0.5,0.95))

# remove tz information so pandas can plot
df2.index = df2.index.tz_localize(None)

df1_comcat = df_comcat.groupby(pd.Grouper(freq='W'))['counts'].agg(['sum'])
df1_comcat['obs_cum_sum'] = df1_comcat['sum'].cumsum()
df1_comcat.index = df1_comcat.index.tz_localize(None)

df2.columns = ["_".join(x) for x in df2.columns.ravel()]
df3 = df2.merge(df1_comcat, left_index=True, right_on='datetime', left_on='datetime')

In [19]:
# plot using matplotlib directly
fig, ax = plt.subplots()

# date formatting
locator = mdates.MonthLocator()  # every month
fmt = mdates.DateFormatter('%b')

ax.plot(df3.index, df3['obs_cum_sum'], color='black', label=comcat.name + ' (Obs)')
ax.plot(df3.index, df3['cum_sum_50%'], color='blue', label=filtered[0].name)
ax.fill_between(df3.index, df3['cum_sum_5%'], df3['cum_sum_95%'], color='blue', alpha=0.2, label='5%-95%')
ax.legend(loc='lower right')
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(fmt)
ax.set_xlabel(df3.index.year.max())
ax.set_ylabel('Cumulative Event Count')


fig.savefig('ucerf3_mwfilt_3.95.pdf', box_inches='tight')
plt.close(fig)